# 벡터 DB : Chroma vs Pinecone
- Chroma : 인메모리 vector DB, 로컬메모리 vector DB
- Pinecone : 클라우드 vector DB
    (Pinecone console에 api key 생성 -> .env (PINECONE_API_KEY등록)

# 0. 패키지 설치

In [ ]:
%pip install -q pinecone-client langchain-pinecone

# 1. Knowledge Base 구성을 위한 데이터 생성

In [1]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
loader = Docx2txtLoader('./tax_docs/with_table.docx')
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200
)
document_list = loader.load_and_split(text_splitter=text_splitter)

In [2]:
len(document_list)

225

In [4]:
# embedding : upstage embedding-query
# https://python.langchain.com/v0.2/docs/integrations/text_embedding/upstage
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings
load_dotenv()
embedding = UpstageEmbeddings(
    model="solar-embedding-1-large"
    # model="embedding-query"
)

In [5]:
%%time
# pinecone vector database
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

pc = Pinecone()
# 데이터를 처음 업로드할 때
index_name = "tax-index-upstage"
# database = PineconeVectorStore.from_documents(
#     documents=document_list,
#     embedding=embedding,
#     index_name=index_name
# )
# 업로드한 벡터DB 가져올 때
database = PineconeVectorStore(
    embedding=embedding, # 질문을 임베딩하여 유사도 검색
    index_name=index_name
)

CPU times: total: 141 ms
Wall time: 1.12 s


# 2. 제공되는 prompt를 활용하여 답변 생성

In [6]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4.1-nano")

In [7]:
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt":prompt}
)

In [8]:
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"
ai_message = qa_chain.invoke({'query':query})
ai_message

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': '연봉 5천만원인 직장인의 소득세 금액은 명확히 제시되지 않았습니다. 그러나 일반적으로 소득세는 총급여액과 과세표준에 따라 차등 적용되며, 해당 정보만으로 정확한 세액을 계산하기 어렵습니다. 따라서, 구체적인 세율과 공제액을 고려해야 하며, 정확한 금액은 세무 전문가나 세금 계산기를 이용하는 것이 좋습니다.'}